In [1]:
import netCDF4
import numpy as np

from datetime import datetime,timedelta
import math

In [2]:
pi=math.pi
#你想要投影的中心點位置 
#lon
rlambda0 = 128.09
#lat
phi0 = 21.239
rlambda0=rlambda0*pi/180
phi0=phi0*pi/180

# particle tracking 位置
x=119.14
y=25.02


In [ ]:
#尋找對應的位置，你的點為在hycom資料的位置
def find_loc(x,cpp_x):
    for st,i in enumerate(cpp_x):
        if(x<=i):
            break
    return st-1,st

In [ ]:
#投影座標轉換，將經緯度轉換成公尺單位
def cpp(ifl,xin,yin,rlambda0,phi0):
    r=6378206.4
    pi=3.1415926
    if(ifl==1):
        xin=xin*pi/180
        yin=yin*pi/180
        xout=r*(xin-rlambda0)*np.cos(phi0)
        yout=yin*r
    elif(ifl==-1):
        xout=rlambda0+xin/r/np.cos(phi0)
        yout=yin/r
        xout=xout/pi*180
        yout=yout/pi*180
    return xout,yout


In [ ]:
#差分公式
def power(x,y,cpp_x,cpp_y,u):
    xloc1,xloc2=find_loc(x,cpp_x)
    yloc1,yloc2=find_loc(y,cpp_y)
    x1=cpp_x[xloc1]
    x2=cpp_x[xloc2]
    y1=cpp_y[yloc1]
    y2=cpp_y[yloc2]
    u1=u[yloc1,xloc1]
    u2=u[yloc1,xloc2]

    if(np.isnan(u1)):
        ans1=u2
    elif(np.isnan(u2)):
        ans1=u1
    elif( (np.isnan(u1)) & (np.isnan(u2)) ):
        ans2=0
    else:
        ans1=u1-((u1-u2)/(x1-x2)*(x1-x))

    u1=u[yloc2,xloc1]
    u2=u[yloc2,xloc2]
    if(np.isnan(u1)):
        ans2=u2
    elif(np.isnan(u2)):
        ans2=u1
    elif( (np.isnan(u1)) & (np.isnan(u2)) ):
        ans2=0
    else:
        ans2=u1-((u1-u2)/(x1-x2)*(x1-x))

    if(ans1==0):
        ans=ans2
    elif(ans2==0):
        ans=ans1
    else:
        ans=ans1-((ans1-ans2)/(y1-y2)*(y1-y))
    return ans


In [ ]:
#檢查一下是否有跑出邊界 ，把點為拉回來的意思
def check(x,y,cpp_x,cpp_y,u,st):
    xloc1,xloc2=find_loc(x,cpp_x)
    yloc1,yloc2=find_loc(y,cpp_y)
    x1=cpp_x[xloc1]
    x2=cpp_x[xloc2]
    y1=cpp_y[yloc1]
    y2=cpp_y[yloc2]

    u1=u[yloc1,xloc1]
    u2=u[yloc1,xloc2]
    u3=u[yloc2,xloc1]
    u4=u[yloc2,xloc2]

    if(np.isnan(u1)):
        if((y2-y)>(x2-x)):
            x=x2
        else:
            y=y2
    if(np.isnan(u2)):
        if((y2-y)>(x-x1)):
            x=x1
        else:
            y=y2
    if(np.isnan(u3)):
        if((y-y1)>(x2-x)):
            x=x2
        else:
            y=y1
    if(np.isnan(u4)):
        if((y-y1)>(x-x1)):
            x=x1
        else:
            y=y1
    return x,y


In [ ]:
#計算下一點跑去哪裡了，小心一點變數名稱可能不一樣
def go_count(x,y,dataname,time):
    data = netCDF4.Dataset(dataname)
    lon=data['lon'][:].data
    lat=data['lat'][:].data
    cpp_x=[]
    cpp_y=[]
    for i,j in zip(lon,lat):
        a,b=cpp(1,i,j,rlambda0,phi0)
        cpp_x.append(a)
        cpp_y.append(b)

    v_data=data['water_v'][time,:,:].data
    u_data=data['water_u'][time,:,:].data


    ans_lon=[]
    ans_lat=[]

    for st,i in enumerate(range(24)):
        u=power(x,y,cpp_x,cpp_y,u_data)
        #print(u)
        v=power(x,y,cpp_x,cpp_y,v_data)
        #print(v)
        x=x+u*3600
        y=y+v*3600
        x,y=check(x,y,cpp_x,cpp_y,v_data,st)
        ans_lon.append(x)
        ans_lat.append(y)

    return ans_lon,ans_lat


In [ ]:
#計算
def particle(x,y,all_name):
    ans_lon=[]
    ans_lat=[]
    for st,i in enumerate(all_name):
        lon,lat=go_count(x,y,i,st)
        ans_lon.extend(lon)
        ans_lat.extend(lat)
        x=ans_lon[-1]
        y=ans_lat[-1]
    return ans_lon,ans_lat


In [ ]:
x,y=cpp(1,x,y,rlambda0,phi0)




#需要修改你的檔案名稱唷!!!
all_name=[]
for d in range(61):
    all_name.append('vel_ave.nc')



ans_lon,ans_lat=particle(x,y,all_name)


with open('info','w') as f:
    for x,y in zip(ans_lon,ans_lat):
        a,b=cpp(-1,x,y,rlambda0,phi0)
        f.writelines(str('%.8f'% a)+"  "+str('%.8f'% b))
        f.writelines('\n')
